In [1]:
import os

In [2]:
%pwd

'c:\\Users\\manoj\\Desktop\\kidney\\kidney\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\manoj\\Desktop\\kidney\\kidney'

In [ ]:
import dagshub
dagshub.init(repo_owner='seenusharma2022', repo_name='kidney', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Note: you may need to restart the kernel to use updated packages.


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4b40e92d-3839-4ecc-9298-a0f959c4564c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5abfb17fbeedbf2692af652e8688b84d244ac921da91971fab3ffe7a1d58b7ac


[2026-01-24 12:09:41,661: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2026-01-24 12:09:42,515: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2026-01-24 12:09:43,909: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as seenusharma2022

...

Initialized MLflow to track repo "seenusharma2022/kidney"

[2026-01-24 12:09:46,584: INFO: helpers: Initialized MLflow to track repo "seenusharma2022/kidney"]


Repository seenusharma2022/kidney initialized!

[2026-01-24 12:09:46,584: INFO: helpers: Repository seenusharma2022/kidney initialized!]


c:\Users\manoj\anaconda3\envs\kidney310\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [13]:
import tensorflow as tf

In [14]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [18]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/seenusharma2022/kidney.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [21]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [22]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [23]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-24 12:22:38,353: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-24 12:22:38,376: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-24 12:22:38,386: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 19s 2s/step - loss: 0.2477 - accuracy: 0.9568
[2026-01-24 12:22:59,535: INFO: common: json file saved at: scores.json]


2026/01/24 12:23:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-24 12:23:06,853: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\manoj\AppData\Local\Temp\tmpo5htguqm\model\data\model\assets
[2026-01-24 12:23:08,102: INFO: builder_impl: Assets written to: C:\Users\manoj\AppData\Local\Temp\tmpo5htguqm\model\data\model\assets]


2026/01/24 12:23:38 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'backports-tarfile'}
c:\Users\manoj\anaconda3\envs\kidney310\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2026/01/24 12:25:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


NameError: name 'train_generator' is not defined